# Classificação com Decision Trees
## ELE606 - Tópicos Especiais em Inteligência Artificial

> Aluno: [Pedro Artur Varela](https://github.com/pedro-varela1)

<img src="https://i1.wp.com/cloudmark.github.io/images/kotlin/ID3.png"
    style="width:500px; float: right; margin: 0 40px 40px 40px;"></img>
  
![divider](https://webstockreview.net/images/divider-clipart-design-line-5.png)

## Introdução

Árvores de Decisão (DTs) são um método de aprendizagem supervisionado não paramétrico usado para classificação e regressão. O objetivo é criar um modelo que preveja o valor de um variável alvo aprendendo regras de decisão simples inferidas a partir dos dados Características. Uma árvore pode ser vista como uma aproximação constante por partes.

---

## Algoritmo e código

### Pré-processamento dos dados

Primeiramente, vamos importar os módulos utilizados.

In [ ]:
!pip install tensorflow_decision_forests

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf   # Modelos de IA e trabalho com tensores
import tensorflow_decision_forests as tfdf    # Modelo Florestas de Decisão - Árvores de Decisão
import pandas as pd   # Manipulação dos dados
import seaborn as sns   # Visualização
import matplotlib.pyplot as plt   # Visualização
import numpy as np    # Manipulação Numérica
from sklearn.datasets import load_wine    # Carregar dataset

Vamos carregar a base de dados na configuração desejada (esse passo realizamos na atividade anterior).

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
heart_df = pd.read_csv('../input/heart-disease-dataset/heart.csv')
heart_df.head()

Vamos carregar a base de dados na configuração desejada (esse passo realizamos na atividade anterior).

In [ ]:
heart_df.info()  # Informations

In [ ]:
heart_df.describe() # Descricao

In [ ]:
# Ver mapa de calor mostrando a correlação entre as variáveis

corr = heart_df.corr()   # Retirar correlação

# Gerar máscara para o gráfico - será triangular
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

# Gerar um colormap diversificado
cmap = sns.diverging_palette(220, 10, as_cmap=True)

with sns.axes_style("white"):
    # Gerar a figura com o matplotlib
    f, ax = plt.subplots(figsize=(10, 8))
    ax = sns.heatmap(corr, cmap=cmap, mask=mask, vmax=.3, square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

Vemos que a correlação de _chol_ e _fbs_ com o _target_ é muito pequena, vamos retirar essa coluna da análise.

In [ ]:
heart_df.drop(['chol', 'fbs'], axis = 1, inplace = True)
heart_df.head()

Vamos separar os dados de treinamento e os dados de teste.

In [ ]:
def split_dataset(dataset, test_ratio=0.30):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, valid_ds_pd = split_dataset(heart_df)
print("{} examples in training, {} examples in testing.".format(
    len(train_ds_pd), len(valid_ds_pd)))

Agora, para manipular o modelo com o Tensorflow, é necessário converter os dados de pandas dataframe para tensor.

In [ ]:
label = 'target'

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)

valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, label=label)

Vamos construir o modelo da Floresta de Decisão, com os parâmetros padrão do Tensorflow..

In [ ]:
rf = tfdf.keras.RandomForestModel(num_trees=200)
rf.compile(metrics=["accuracy", "mse"])

### Treinamento do Modelo

O modelo já está projeto, agora basta treiná-lo.

In [ ]:
rf.fit(x=train_ds)

Vamos ver o formato da árvore de decisão de índice 0.

In [ ]:
tfdf.model_plotter.plot_model_in_colab(rf, tree_idx=0, max_depth=3)

### Análise dos Resultados

Vamos analisar os resultados através do gráfico da acurácia pelo número de árvores.

In [ ]:
logs = rf.make_inspector().training_logs()

plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy")
plt.show()

Vamos ter uma visão geral da acurácia final.

In [ ]:
evaluation = rf.evaluate(x=valid_ds,return_dict=True)

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")